Let's try some basic tests of `VB.jl`:

In [1]:
include("VB.jl")
using VB
using Distributions

# Make a node

In [2]:
μ = rand(5, 5)
σ = rand(5, 5)
aa = RandomNode(:a, [:i, :j], Normal, μ, σ)
bb = RandomNode(:b, [:j, :k], Gamma, rand(5, 3), rand(5, 3))

VB.RandomNode{Distributions.Gamma}(:b,[:j,:k],5x3 Array{Distributions.Gamma,2}:
 Distributions.Gamma(α=0.6835825409163594, θ=0.8302434075817287)    …  Distributions.Gamma(α=0.5374911369304947, θ=0.3911655757982906) 
 Distributions.Gamma(α=0.5339856582890734, θ=0.9673379525374954)       Distributions.Gamma(α=0.5746172372406446, θ=0.42086587965651656)
 Distributions.Gamma(α=0.3845902716593208, θ=0.04316228514278886)      Distributions.Gamma(α=0.7030116194386717, θ=0.5579484513988313) 
 Distributions.Gamma(α=0.7762307915066216, θ=0.7260466602168107)       Distributions.Gamma(α=0.7025900648946914, θ=0.824479320512072)  
 Distributions.Gamma(α=0.9730315925117083, θ=0.005543988674875822)     Distributions.Gamma(α=0.1055892432007155, θ=0.3141398998744296) )

In [3]:
cc = ConstantNode(rand(3, 4), [:i, :j])

VB.ConstantNode{Float64}(symbol("##const#7589"),[:i,:j],3x4 Array{Float64,2}:
 0.105611  0.371904  0.63345    0.6777   
 0.126563  0.603875  0.039767   0.611282 
 0.244376  0.813678  0.0800067  0.0887114)

In [4]:
nodes = Node[aa, bb]
fi = get_structure(nodes...)
fi

VB.FactorInds([:i,:j,:k],[5,3,5],Dict(:a=>[1,2],:b=>[2,3]))

In [5]:
dump(:([i, j, k]))

Expr 

In [6]:
dump(:(a[i, j] ~ B[x]))

# Make a factor

In [2]:
dims = (10, 2)
μ = RandomNode(:μ, [:j], Normal, rand(dims[2]), ones(dims[2]))
τ = RandomNode(:τ, [:scalar], Gamma, 1, 1)
x = RandomNode(:x, [:i, :j], Normal, rand(dims), ones(dims))
f = @factor LogNormalFactor x μ τ

VB.LogNormalFactor{3}(VB.FactorInds([:i,:j,:scalar],[10,2,1],Dict(:τ=>[3],:μ=>[2],:x=>[1,2])),VB.RandomNode{Distributions.Normal}(:x,[:i,:j],10x2 Array{Distributions.Normal,2}:
 Distributions.Normal(μ=0.24066657971648442, σ=1.0)  …  Distributions.Normal(μ=0.1376198246566982, σ=1.0)  
 Distributions.Normal(μ=0.37644750877443056, σ=1.0)     Distributions.Normal(μ=0.3996162277927855, σ=1.0)  
 Distributions.Normal(μ=0.9641029723202366, σ=1.0)      Distributions.Normal(μ=0.9987506888688007, σ=1.0)  
 Distributions.Normal(μ=0.03479771894599404, σ=1.0)     Distributions.Normal(μ=0.9580003289950512, σ=1.0)  
 Distributions.Normal(μ=0.32676780986611087, σ=1.0)     Distributions.Normal(μ=0.6068062541292154, σ=1.0)  
 Distributions.Normal(μ=0.44437861702291204, σ=1.0)  …  Distributions.Normal(μ=0.140598030358138, σ=1.0)   
 Distributions.Normal(μ=0.8210015353830207, σ=1.0)      Distributions.Normal(μ=0.8329649939648256, σ=1.0)  
 Distributions.Normal(μ=0.31395905333350393, σ=1.0)     Distributio

In [25]:
macroexpand(:(@factor LogNormalFactor x μ τ))

quote  # /Users/jmxp/code/vbgraph/types.jl, line 41:
    fi = get_structure(x,μ,τ) # /Users/jmxp/code/vbgraph/types.jl, line 42:
    LogNormalFactor{length(fi.indices)}(fi,x,μ,τ)
end

In [27]:
project(f, :x, (3, 2, 5, 7, 9))

Distributions.Normal(μ=0.5215420301621809, σ=1.0)

# Make a (generated) value function

In [3]:
value(f)

-1.6364820211834816

In [4]:
g = EntropyFactor(x)
value(g)

-1.537827013900917

In [5]:
naturals(x)

(31.131686486597527,-185.00519645896833)

# Let's make a simple model

We want a simple model of inference for a normal distribution:

$$
\begin{aligned}
    y \sim \mathcal{N}(\mu, \tau) \\
    \mu \sim \mathcal{N}(\mu_0, \tau_0) \\
    \tau \sim \mathrm{Ga}(\alpha, \beta) \\
    q(\mu) = \mathcal{N}(m, t) \\
    q(\tau) = \mathrm{Ga}(a, b)
\end{aligned}
$$

This model is conjugate, and can be solved by straightforward updates of the natural parameters of the posterior.

## Make the nodes

We need to make nodes for all the random variables that will need to be updated (we can also create nodes for parameter arrays, but these will be converted automatically if we don't).

In [12]:
dims = (5, 6)

# note: it won't matter much how we initialize here
μ = NodeArray(Normal, zeros(dims), ones(dims))
τ = NodeArray(Gamma, ones(dims), ones(dims))

nobs = 20
y = rand(nobs, dims...);

Now make factors: We need a Normal factor for the observation model plus a prior and an entropy for each node.

In [13]:
obs = LogNormalFactor(y, μ, τ)
μ_prior = LogNormalFactor(μ, zeros(dims), 2 * ones(dims))
τ_prior = LogGammaFactor(τ, 1.1 * ones(dims), ones(dims))

VB.LogGammaFactor(5x6 Array{Distributions.Gamma,2}:
 Distributions.Gamma(α=1.0, θ=1.0)  …  Distributions.Gamma(α=1.0, θ=1.0)
 Distributions.Gamma(α=1.0, θ=1.0)     Distributions.Gamma(α=1.0, θ=1.0)
 Distributions.Gamma(α=1.0, θ=1.0)     Distributions.Gamma(α=1.0, θ=1.0)
 Distributions.Gamma(α=1.0, θ=1.0)     Distributions.Gamma(α=1.0, θ=1.0)
 Distributions.Gamma(α=1.0, θ=1.0)     Distributions.Gamma(α=1.0, θ=1.0),5x6 Array{Float64,2}:
 1.1  1.1  1.1  1.1  1.1  1.1
 1.1  1.1  1.1  1.1  1.1  1.1
 1.1  1.1  1.1  1.1  1.1  1.1
 1.1  1.1  1.1  1.1  1.1  1.1
 1.1  1.1  1.1  1.1  1.1  1.1,5x6 Array{Float64,2}:
 1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0)

In [21]:
value(obs), value(μ_prior), value(τ_prior)

(-38.35125483195037,-67.96536370453937,7.496173237795199)

In [22]:
m = VBModel(Dict(:μ => μ, :τ => τ), [obs, μ_prior, τ_prior])

VB.VBModel(Dict{Symbol,VB.Node}(:τ=>VB.RandomNode{Distributions.Gamma,2}(5x6 Array{Distributions.Gamma,2}:
 Distributions.Gamma(α=1.0, θ=1.0)  …  Distributions.Gamma(α=1.0, θ=1.0)
 Distributions.Gamma(α=1.0, θ=1.0)     Distributions.Gamma(α=1.0, θ=1.0)
 Distributions.Gamma(α=1.0, θ=1.0)     Distributions.Gamma(α=1.0, θ=1.0)
 Distributions.Gamma(α=1.0, θ=1.0)     Distributions.Gamma(α=1.0, θ=1.0)
 Distributions.Gamma(α=1.0, θ=1.0)     Distributions.Gamma(α=1.0, θ=1.0),Dict{VB.Factor,Symbol}(VB.LogGammaFactor(VB.RandomNode{Distributions.Gamma,2}(#= circular reference =#),VB.ConstantNode{Float64,2}(5x6 Array{Float64,2}:
 1.1  1.1  1.1  1.1  1.1  1.1
 1.1  1.1  1.1  1.1  1.1  1.1
 1.1  1.1  1.1  1.1  1.1  1.1
 1.1  1.1  1.1  1.1  1.1  1.1
 1.1  1.1  1.1  1.1  1.1  1.1,Dict{VB.Factor,Symbol}()),VB.ConstantNode{Float64,2}(5x6 Array{Float64,2}:
 1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0,

In [23]:
m.nodes[:μ].factormap

Dict{VB.Factor,Symbol} with 2 entries:
  VB.LogNormalFactor(VB.C… => :μ
  VB.LogNormalFactor(VB.R… => :x

In [24]:
naturals(obs, Val{:x}, μ)

(
5x6 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0,

5x6 Array{Float64,2}:
 -0.5  -0.5  -0.5  -0.5  -0.5  -0.5
 -0.5  -0.5  -0.5  -0.5  -0.5  -0.5
 -0.5  -0.5  -0.5  -0.5  -0.5  -0.5
 -0.5  -0.5  -0.5  -0.5  -0.5  -0.5
 -0.5  -0.5  -0.5  -0.5  -0.5  -0.5)

In [25]:
naturals(obs, Val{:μ}, μ)

(
5x6 Array{Float64,2}:
 0.019526  0.64192   0.920968   0.221961  0.497111  0.376147
 0.645838  0.886014  0.635094   0.86881   0.173711  0.234695
 0.571483  0.605633  0.240604   0.252273  0.424832  0.433595
 0.246642  0.580497  0.81434    0.503185  0.399902  0.499845
 0.8564    0.315123  0.0795404  0.555742  0.340429  0.753323,

5x6 Array{Float64,2}:
 -0.5  -0.5  -0.5  -0.5  -0.5  -0.5
 -0.5  -0.5  -0.5  -0.5  -0.5  -0.5
 -0.5  -0.5  -0.5  -0.5  -0.5  -0.5
 -0.5  -0.5  -0.5  -0.5  -0.5  -0.5
 -0.5  -0.5  -0.5  -0.5  -0.5  -0.5)

In [26]:
naturals(obs, Val{:τ}, τ)

(0.5,
5x6 Array{Float64,2}:
 0.500191  0.706031  0.924091  0.524633  0.62356   0.570743
 0.708553  0.89251   0.701672  0.877416  0.515088  0.527541
 0.663296  0.683396  0.528945  0.531821  0.590241  0.594002
 0.530416  0.668488  0.831575  0.626598  0.579961  0.624922
 0.86671   0.549651  0.503163  0.654425  0.557946  0.783748)

In [27]:
method_exists(naturals, (LogNormalFactor, Type{Val{:x}}, RandomNode{Normal}))

true

In [28]:
check_conjugate(m.nodes[:τ])

true

In [29]:
μ.data

5x6 Array{Distributions.Normal,2}:
 Distributions.Normal(μ=0.0, σ=1.0)  …  Distributions.Normal(μ=0.0, σ=1.0)
 Distributions.Normal(μ=0.0, σ=1.0)     Distributions.Normal(μ=0.0, σ=1.0)
 Distributions.Normal(μ=0.0, σ=1.0)     Distributions.Normal(μ=0.0, σ=1.0)
 Distributions.Normal(μ=0.0, σ=1.0)     Distributions.Normal(μ=0.0, σ=1.0)
 Distributions.Normal(μ=0.0, σ=1.0)     Distributions.Normal(μ=0.0, σ=1.0)

In [30]:
update!(μ, Val{:conjugate})

In [31]:
μ.data

5x6 Array{Distributions.Normal,2}:
 Distributions.Normal(μ=-0.33974642587876164, σ=0.7071067811865476)  …  Distributions.Normal(μ=-0.08757735964789944, σ=0.7071067811865476)   
 Distributions.Normal(μ=0.1031230083822061, σ=0.7071067811865476)       Distributions.Normal(μ=-0.18759892024976002, σ=0.7071067811865476)   
 Distributions.Normal(μ=0.05054604661844582, σ=0.7071067811865476)      Distributions.Normal(μ=-0.046955363717725414, σ=0.7071067811865476)  
 Distributions.Normal(μ=-0.17915149839257358, σ=0.7071067811865476)     Distributions.Normal(μ=-0.00010971345274074199, σ=0.7071067811865476)
 Distributions.Normal(μ=0.2520128176967853, σ=0.7071067811865476)       Distributions.Normal(μ=0.17912620274862429, σ=0.7071067811865476)    